In [ ]:
import pandas as pd

In [ ]:
import pywikibot as pb
site = pb.Site("wikidata", "wikidata")
repo = site.data_repository()

In [ ]:
!pip install xlrd

In [ ]:
!pip install XlsxWriter

In [ ]:
def create_item(site, label_dict):
    new_item = pb.ItemPage(site)
    new_item.editLabels(labels=label_dict, summary="Creating Red List item")
    return new_item

In [ ]:
def set_claim(item, property, string_data):
    clm=new_item.get()['claims'][property][0]
    clm_trgt = clm.getTarget()
    print(clm_trgt)
    if clm_trgt != string_data :
        claim = pb.Claim(repo, property)
        claim.setTarget(string_data)
        item.addClaim(claim, bot=False, summary="Test")
        return claim
    else :
        return None

In [ ]:
red_list_df = pd.read_excel("red2.xlsx")

In [ ]:
red_list_df.head()

In [ ]:
output_df = pd.DataFrame(columns=red_list_df.columns)
for index, row in red_list_df.iterrows():
    if index > 4:
        break
    code = row['URL'].split('/')[-2]
    print (code)
    if pd.isnull(row['Wikidata ID']):
        labels = {"es": row['Nombre']}
        print (labels)
        new_item = create_item(site, labels)
        new_item_id = new_item.getID()
    else :
        new_item = pb.ItemPage(repo, row['Wikidata ID'])
    new_claim = set_claim(new_item, 'P4868', row['URL'].split('/')[-2])
    #row['Wikidata ID'] = new_item_id
    #output_df = output_df.append(row, ignore_index=True)

In [ ]:
writer = pd.ExcelWriter('simple.xlsx', engine='xlsxwriter')
output_df.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [ ]:
new_item = pb.ItemPage(site, 'Q52120504')
clm_list=new_item.get()['claims']['P4868']
for clm in clm_list:
    clm_trgt = clm.getTarget()
    print(clm_trgt)